In [12]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import torchmetrics

from utils.train_utils import *
from model.unet import UNet16


import warnings
warnings.filterwarnings('ignore')

In [13]:

save_dir = '/home/jovyan/G1/dragon_ball/utils'

In [14]:

import argparse
def parse_args():
	parser = argparse.ArgumentParser(description='CRACK SEGMENTATION')
	parser.add_argument('--batch_size', '-b', type=int, default=4)
	parser.add_argument('--train', '-t', default=True)
	parser.add_argument('--project_name', '-n', default='crack_seg')
	parser.add_argument('--seed', type=int, default=11)
	parser.add_argument('--save_model', '-s', action='store_true', default=False)

	return parser.parse_args()


def check_mkdir(dir_name):
	if not os.path.exists(dir_name):
		os.mkdir(dir_name)


In [42]:

def train(train_loader, model, optimizer, criterion, logger):
	metrics = 0
	model.train()
	losses = []
	
	for batch_idx, (image, label) in enumerate(train_loader):
		print(batch_idx)
		optimizer.zero_grad()
		# Forward 
		output = model(image)
		# loss_func
		loss = criterion(output, label)
		# metrics	#
		losses.append(loss)
		logger.info(loss)
		#			#


		# Gradinet 
		loss.backward()   
		# weight update 
		optimizer.step() 
	#
	return metrics, losses

def test(test_loader, model, criterion, logger):
	metrics = 0
	model.eval()
	losses = []

	with torch.no_grad():
		for batch_idx, (image, label) in enumerate(test_loader):
			# Forward 
			outputs = model(image) 
			# loss_func 
			loss = criterion(outputs, label)
			# metrics	#
			losses.append(loss)
			# accuracy = torchmetrics.Accuracy()
			# preds = torch.argmax(loss, dim=1)
			# print(accuracy(preds, label))
			#			#
	return metrics, losses


In [40]:

from data.Dataset import CrackDataset
# parser = argparse.ArgumentParser(description='CRACK SEGMENTATION')
# parser.add_argument('--batch_size', '-b', type=int, default=4)
# parser.add_argument('--train', '-t', default=True)
# parser.add_argument('--project_name', '-n', default='crack_seg')
# parser.add_argument('--seed', type=int, default=11)
# parser.add_argument('--save_model', '-s', action='store_true', default=False)
# args = parse_args()

seed = 11
save_model = False
project_name = 'crack_seg'
no_cuda = True

#
lr = 0.001
momentum = 0.9
weight_decay = 1e-4
batch_size = 4
num_workers = 4
epoches = 1 # 50
batch_size, num_workers = 16,4

torch.manual_seed(seed)

#
print(torch.__version__, torch.cuda.__path__)	
use_cuda = not no_cuda and torch.cuda.is_available()
# device_id = int(os.environ["RANK"])
# print(os.environ)

#GPU 갯수 확인
AVAIL_GPUS = max(1, torch.cuda.device_count())
print(AVAIL_GPUS)

1.12.0+cu116 ['/home/jovyan/.venv/torch1.10.0-py3.8-cuda11.3/lib/python3.8/site-packages/torch/cuda']
2


In [43]:

path = create_save_dir(project_name)
print(path)

logger = get_log(path)

#
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("gpu") if torch.cuda.is_available() else print("cpu")

#
train_loader = CrackDataset(split='train')
test_loader = CrackDataset(split='test')

#
model = UNet16(pretrained=False)

#
optimizer = torch.optim.SGD(model.parameters(), lr,
								momentum=momentum,
								weight_decay=weight_decay)

#
criterion = torch.nn.BCEWithLogitsLoss().to('cuda')

if save_model:
	torch.save(model.state_dict())

for epoch in range(epoches):
	# train
	metrics, loss = train(train_loader, model, optimizer, criterion, logger)

	#test
	test_metrics, test_loss = test(test_loader, model, criterion, logger)

	if epoch == epoches-1:
		print(epoch)
	

result/crack_seg/
gpu
0


In [28]:
print(metrics, test_metrics)
print(loss, test_loss)

0 0
[] []


In [45]:
logger.info("1")

2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1
2022-10-17 13:29:17,663 - 1


In [39]:
logger.info("loss testing"+str(19))

2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
2022-10-17 13:07:10,159 - loss testing19
